<a href="https://colab.research.google.com/github/INA-95/spotify_api/blob/main/Spotify_api_231022.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# library

import requests
import base64
import pandas as pd

In [102]:
# function

# authorization

def get_access_token(client_id:str, client_secret:str) -> str:
    auth_url = 'https://accounts.spotify.com/api/token'
    auth_header = {
        'Authorization': f'Basic {base64.b64encode((client_id + ":" + client_secret).encode()).decode()}'
    }

    auth_data = {
        'grant_type': 'client_credentials',
        'client_id' : client_id,
        'client_secret' : client_secret
    }

    auth_response = requests.post(auth_url, data = auth_data, headers = auth_header)
    auth_response_data = auth_response.json()
    access_token = auth_response_data['access_token']
    return access_token


# get your playlists

def get_playlists(access_token:str, user_name:str) -> dict:
    headers = {"Authorization" : f"Bearer {access_token}"}
    res = requests.get(
        f"https://api.spotify.com/v1/users/{user_name}/playlists",
        headers = headers
    )

    playlist_array = res.json()['items']
    playlist_dict = {playlist['id']:playlist['name'] for playlist in playlist_array}

    return playlist_dict


# get your tracks (playlist > track)

def get_playlist_tracks(access_token:str, playlist_dict:list[str], user_name:str) -> dict:
    headers = {"Authorization": f"Bearer {access_token}"}
    offset = 0

    tracks = []
    playlist_ids = [playlist_id for playlist_id in playlist_dict.keys()]

    for playlist_id in playlist_ids:

        res = requests.get(
            f"https://api.spotify.com/v1/users/{user_name}/playlists/{playlist_id}/tracks?offset={offset}",
            headers=headers
        )
        track = res.json()
        tracks.append(track)

    return tracks

# get artist, title, track_id

def get_track_info(tracks:list[dict[str]]) -> pd.DataFrame:

    artists = []
    titles = []
    track_ids = []

    for track in tracks:
        for val in track['items']:
            track_info = val.get('track')

            artist = track_info.get('artists')
            artist = artist[0]['name']

            title = track_info.get('name')
            track_id = val.get('track')['uri'].split(':')[-1]

            artists.append(artist)
            titles.append(title)
            track_ids.append(track_id)

        df = pd.DataFrame(list(zip(titles, artists, track_ids)),
                        columns = ['title', 'artist', 'track_id'])

    return df


# get your tracks audio feature

def tracks_audio_features(access_token:str, track_ids:list[str]) -> pd.DataFrame:

    feature_lst = []

    for track_id in track_ids:
        headers = {"Authorization" : f"Bearer {access_token}"}
        res = requests.get(
            f"https://api.spotify.com/v1/audio-features/{track_id}",
            headers = headers
        )
        feature = res.json()

        feature_lst.append(feature)
        df = pd.DataFrame.from_dict(feature_lst)

    return df